---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [67]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [68]:
def get_list_of_university_towns():
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    #Read the file
    states_towns_raw = pd.read_table('university_towns.txt',header=None).rename(columns={0: 'Raw'})
    ##Create an empty dataframe
    col_names =  ['State', 'RegionName']
    states_towns  = pd.DataFrame(columns = col_names)

    ##Repeat the state (which ends with "[edit]"), then insert each state,region as a row in the empty dataframe
    State = None
    for region in states_towns_raw['Raw']:
        if region[-6:] == "[edit]":
            State = region[:-6]
        else:
            states_towns = states_towns.append({'State': State,'RegionName': region.split(' (')[0]},ignore_index=True)
    return states_towns

In [69]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_yearly = pd.read_excel('gdplev.xls',skiprows=78,skipfooter=191)
    GDP_yearly = GDP_yearly.drop(['Unnamed: 3', '1964q3',692.8,3766.9,'Unnamed: 7'], axis=1)
    GDP_yearly.columns = ['year', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

    GDP_quar = pd.read_excel('gdplev.xls',skiprows=219)
    GDP_quar = GDP_quar.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 7'], axis=1)
    GDP_quar.columns = ['yearQuar', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

    GDP_quar['PrevGdp'] = GDP_quar['GDP in billions of current dollars'].shift(1)
    GDP_quar['NxtGdp'] = GDP_quar['GDP in billions of current dollars'].shift(-1)
    GDP_quar['flag'] = (GDP_quar['GDP in billions of current dollars'] - GDP_quar['PrevGdp']) > 0
    GDP_quar['NxtFlag'] = GDP_quar['flag'].shift(-1)
    GDP_quar['NxtDoubleFlag'] = GDP_quar['flag'].shift(-2)
    GDP_quar['PrvFlag'] = GDP_quar['flag'].shift(1)
    GDP_quar['PrvDoubleFlag'] = GDP_quar['flag'].shift(2)
    GDP_quar['PrvTripleFlag'] = GDP_quar['flag'].shift(3)
    GDP_quar['PrvQuadFlag'] = GDP_quar['flag'].shift(4)
    GDP_quar['StartRecc'] = GDP_quar.apply(lambda x:1 if (x['NxtDoubleFlag'] == False and x['NxtFlag'] == False) else 0, axis = 1)
    GDP_quar['BottomRecc'] = GDP_quar.apply(lambda x:1 if 
                                         (x['NxtDoubleFlag'] == True and x['NxtFlag'] == True and 
                                         x['PrvDoubleFlag'] == False and x['PrvFlag'] == False) else 0, axis = 1)
    GDP_quar['EndRecc'] = GDP_quar.apply(lambda x:1 if 
                                         (x['flag'] == True and x['PrvFlag'] == True and 
                                         x['PrvDoubleFlag'] == False and x['PrvTripleFlag'] == False) else 0, axis = 1)
    
    
    return GDP_quar[GDP_quar['StartRecc'] == 1].iloc[0]['yearQuar']

In [70]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP_yearly = pd.read_excel('gdplev.xls',skiprows=78,skipfooter=191)
    GDP_yearly = GDP_yearly.drop(['Unnamed: 3', '1964q3',692.8,3766.9,'Unnamed: 7'], axis=1)
    GDP_yearly.columns = ['year', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

    GDP_quar = pd.read_excel('gdplev.xls',skiprows=219)
    GDP_quar = GDP_quar.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 7'], axis=1)
    GDP_quar.columns = ['yearQuar', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

    GDP_quar['PrevGdp'] = GDP_quar['GDP in billions of current dollars'].shift(1)
    GDP_quar['NxtGdp'] = GDP_quar['GDP in billions of current dollars'].shift(-1)
    GDP_quar['flag'] = (GDP_quar['GDP in billions of current dollars'] - GDP_quar['PrevGdp']) > 0
    GDP_quar['NxtFlag'] = GDP_quar['flag'].shift(-1)
    GDP_quar['NxtDoubleFlag'] = GDP_quar['flag'].shift(-2)
    GDP_quar['PrvFlag'] = GDP_quar['flag'].shift(1)
    GDP_quar['PrvDoubleFlag'] = GDP_quar['flag'].shift(2)
    GDP_quar['PrvTripleFlag'] = GDP_quar['flag'].shift(3)
    GDP_quar['PrvQuadFlag'] = GDP_quar['flag'].shift(4)
    GDP_quar['StartRecc'] = GDP_quar.apply(lambda x:1 if (x['NxtDoubleFlag'] == False and x['NxtFlag'] == False) else 0, axis = 1)
    GDP_quar['BottomRecc'] = GDP_quar.apply(lambda x:1 if 
                                         (x['NxtDoubleFlag'] == True and x['NxtFlag'] == True and 
                                         x['PrvDoubleFlag'] == False and x['PrvFlag'] == False) else 0, axis = 1)
    GDP_quar['EndRecc'] = GDP_quar.apply(lambda x:1 if 
                                         (x['flag'] == True and x['PrvFlag'] == True and 
                                         x['PrvDoubleFlag'] == False and x['PrvTripleFlag'] == False) else 0, axis = 1)
    
    
    
    return GDP_quar[GDP_quar['EndRecc'] == 1].iloc[0]['yearQuar']

In [71]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP_yearly = pd.read_excel('gdplev.xls',skiprows=78,skipfooter=191)
    GDP_yearly = GDP_yearly.drop(['Unnamed: 3', '1964q3',692.8,3766.9,'Unnamed: 7'], axis=1)
    GDP_yearly.columns = ['year', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

    GDP_quar = pd.read_excel('gdplev.xls',skiprows=219)
    GDP_quar = GDP_quar.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 7'], axis=1)
    GDP_quar.columns = ['yearQuar', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

    GDP_quar['PrevGdp'] = GDP_quar['GDP in billions of current dollars'].shift(1)
    GDP_quar['NxtGdp'] = GDP_quar['GDP in billions of current dollars'].shift(-1)
    GDP_quar['flag'] = (GDP_quar['GDP in billions of current dollars'] - GDP_quar['PrevGdp']) > 0
    GDP_quar['NxtFlag'] = GDP_quar['flag'].shift(-1)
    GDP_quar['NxtDoubleFlag'] = GDP_quar['flag'].shift(-2)
    GDP_quar['PrvFlag'] = GDP_quar['flag'].shift(1)
    GDP_quar['PrvDoubleFlag'] = GDP_quar['flag'].shift(2)
    GDP_quar['PrvTripleFlag'] = GDP_quar['flag'].shift(3)
    GDP_quar['PrvQuadFlag'] = GDP_quar['flag'].shift(4)
    GDP_quar['StartRecc'] = GDP_quar.apply(lambda x:1 if (x['NxtDoubleFlag'] == False and x['NxtFlag'] == False) else 0, axis = 1)
    GDP_quar['BottomRecc'] = GDP_quar.apply(lambda x:1 if 
                                         (x['NxtDoubleFlag'] == True and x['NxtFlag'] == True and 
                                         x['PrvDoubleFlag'] == False and x['PrvFlag'] == False) else 0, axis = 1)
    GDP_quar['EndRecc'] = GDP_quar.apply(lambda x:1 if 
                                         (x['flag'] == True and x['PrvFlag'] == True and 
                                         x['PrvDoubleFlag'] == False and x['PrvTripleFlag'] == False) else 0, axis = 1)
    
    
    
    return GDP_quar[GDP_quar['BottomRecc'] == 1].iloc[0]['yearQuar']

In [72]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    col_list = list(range(6,51))
    housing_data.drop(housing_data.columns[col_list],axis=1,inplace=True)

    ##Turn Dict to Dataframe
    StateDict = pd.DataFrame.from_dict(states , orient = 'index').rename(columns={0:'State'}) 
    Merged_df = housing_data.merge(StateDict,left_on = 'State' , right_index = True)
    Merged_df.drop(['State','RegionID','State_x','Metro','CountyName','SizeRank'],axis=1,inplace = True)
    ##Change order of columns to get State at the begining
    cols = Merged_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    Merged_df = Merged_df[cols]
    ##Change name of columns (that was renamed in the join)
    Merged_df.rename(columns={'State_y': 'State'},inplace = True)

    ##Select the years only
    Years = Merged_df.iloc[:,2:len(Merged_df.columns)]
    ##Convert columns type into datatime
    Years.columns = pd.to_datetime(Years.columns)
    ##Take averages based on quarters
    Years_agg = Years.resample('Q',axis=1).mean()
    ##Convert column names into string
    Years_agg = Years_agg.rename(columns=lambda x: str(x.to_period('Q')).lower())
    ##Get the state and region
    Index_Columns = Merged_df.iloc[:,0:2]
    ##Merge index with agg years
    Final_result = Index_Columns.merge(Years_agg,left_index = True, right_index = True)
    Final_result.set_index(['State','RegionName'],inplace = True)
    return Final_result


'2008q2'

In [188]:
#     Soultion 2: 
#     Desc: The idea was to melt the dataframe and turn the columns of the month into rows and have 1 columns 
#     called years and another called quarater, it didn't work as it requires grouping and some data was duplicated 
#     and the grader wanted them as is

#############################################################
#     housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
#     housing_data
#     col_list = list(range(6,51))
#     housing_data.drop(housing_data.columns[col_list],axis=1,inplace=True)

#     ## Breaking down the coloumns into rows for easy manipulation
#     col_list = list(range(6,len(housing_data.columns)))
#     housing_data_melted = pd.melt(housing_data,id_vars=['State','RegionName'] , value_vars=list(housing_data.columns[col_list]))
#     ##Create anchor variables to group by with (Year, month)
#     housing_data_melted['Year'] = housing_data_melted['variable'].str.slice(0,4)
#     housing_data_melted['Quar'] = housing_data_melted['variable'].apply(lambda x:'q1' if int(x.split('-')[1]) <= 3 else 'q2' if int(x.split('-')[1]) <= 6 else 'q3' if int(x.split('-')[1]) <= 9 else 'q4')
#     housing_data_melted['YearQuar'] = housing_data_melted['Year'] + housing_data_melted['Quar']
#     Aggregatd_df = housing_data_melted.groupby(['State','RegionName','YearQuar'])["value"].agg([np.mean])

#     Aggregatd_df.reset_index(inplace = True)
#     pivoted_df = Aggregatd_df.pivot_table(index=['State','RegionName'], columns='YearQuar', values='mean')
#     pivoted_df.reset_index(inplace = True)
#     pivoted_df.set_index(['State','RegionName'],inplace = True)

In [61]:
# #pivoted_df.reset_index(inplace = True)
# pivoted_df['StateRegion'] = pivoted_df['State'] + pivoted_df['RegionName']
# housing_data['StateRegion'] = housing_data['State'] + housing_data['RegionName']
# len(pivoted_df['StateRegion'].unique()) ## 10592
# len(housing_data['StateRegion'].unique()) ##10592

In [200]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    hdf = convert_housing_data_to_quarters()
    rec_start=get_recession_start()
    rec_bottom = get_recession_bottom()
    rec_end = get_recession_end()
    ul = get_list_of_university_towns()

    QarBeforeRecc = str(pd.to_datetime(rec_start).to_period('Q') - 1).lower()
    hdf['PriceRatio'] = hdf[QarBeforeRecc] / hdf[rec_bottom]

    sublist = (ul.to_records(index=False)).tolist()
    #UniversityTowns = hdf.loc[sublist]
    UniversityTowns = ul.merge(hdf,how='inner',left_on = ['State','RegionName'],right_index = True)
    Non_UniversityTowns = hdf.loc[-hdf.index.isin(sublist)]

    ttest = ttest_ind(UniversityTowns['PriceRatio'],Non_UniversityTowns['PriceRatio'],nan_policy = 'omit')
    universityMean = UniversityTowns['PriceRatio'].mean() 
    Non_UniversityTowns = Non_UniversityTowns['PriceRatio'].mean() 

    better = lambda universityMean,Non_UniversityTowns : 'non-university town' if universityMean > Non_UniversityTowns else 'university town'

    different = lambda x: 'True' if x < 0.01 else 'False' 
    
    Result = (different(ttest.pvalue)== 'True',ttest.pvalue,better(universityMean,Non_UniversityTowns))


    return Result


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:28: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


(True, 0.0027240637047531249, 'university town')

In [192]:
# def test_q6():
#     q6 = run_ttest()
#     different, p, better = q6

#     res = 'Type test: '
#     res += ['Failed\n','Passed\n'][type(q6) == tuple]

#     res += 'Test "different" type: '
#     res += ['Failed\n','Passed\n'][type(different) == bool or type(different) == np.bool_]

#     res += 'Test "p" type: '
#     res += ['Failed\n','Passed\n'][type(p) == np.float64]

#     res +='Test "better" type: '
#     res += ['Failed\n','Passed\n'][type(better) == str]
#     if type(better) != str:
#         res +='"better" should be a string with value "university town" or  "non-university town"'
#         return res
#     res += 'Test "different" spelling: '
#     res += ['Failed\n','Passed\n'][better in ["university town", "non-university town"]]
#     return res
# print(test_q6())

Type test: Passed
Test "different" type: Passed
Test "p" type: Passed
Test "better" type: Passed
Test "different" spelling: Passed



/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:28: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.
